In [3]:
import numpy as np
import pandas as pd
import joblib

In [4]:
# 加载高钾分类数据结构
xgb_J = joblib.load('J_xgboost.joblib')

# 加载铅钡分类数据结构
xgb_Q = joblib.load('Q_xgboost.joblib')

In [5]:
# 加载进行分类预测数据
excel_for_ana = pd.ExcelFile('问题三预测数据处理后.xlsx')

data_J_ana = pd.read_excel(excel_for_ana, "高钾")

data_Q_ana = pd.read_excel(excel_for_ana, "铅钡")

excel_for_ana.close()

In [6]:
# 单纯用于获取J和Q的columns
col_J = pd.read_excel("问题二将亚分类数据.xlsx",sheet_name="高钾权重")
col_Q = pd.read_excel("问题二将亚分类数据.xlsx",sheet_name="铅钡权重")

In [7]:
# 整理好后,用于xgb_J和xgb_Q敏感性分析
data_J_ana_rea = data_J_ana[col_J.columns.values].copy()
data_Q_ana_rea = data_Q_ana[col_Q.columns.values].copy()

In [49]:
data_J_ana_rea

,氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化铜(CuO),氧化钡(BaO)
0,0,0.00000,6.08000,2.11,0
1,0,9.19721,8.20808,1.73,0
2,0,9.95153,6.32273,1.17,0


In [8]:
# 对每个成分逐步
max_J = [3.38,14.52,7.1374988,4.73,10.6,1.97]
max_Q = [7.92, 3.13, 40, 31.23, 4.76887]

In [ ]:
9.3

In [50]:
# 对于一组给出预测结果
def pre_res_J(data):
    return xgb_J.predict(data)

In [57]:
def pre_res_Q(data):
    return xgb_Q.predict(data)

In [68]:
# 给出一组J的结果
# ele给出是什么化学元素
def pre_group_J(num, ele): 
    to_change = np.arange(0, max_J[ele], max_J[ele]/num) + max_J[ele]/num
    res = pd.DataFrame(data=np.zeros((3, num)),columns=to_change)
    for i in range(num):
        data_temp = data_J_ana_rea.copy()
        data_temp.iloc[:,ele] = to_change[i]
        res.iloc[:,i] = pre_res_J(data_temp)
    return res

In [69]:
# 开始得出J的结果

t_J = pd.ExcelWriter("问题三高钾的敏感性分析.xlsx")
for i in range(data_J_ana_rea.shape[1]):
    temp = pre_group_J(20, i)
    temp.to_excel(t_J, data_J_ana_rea.columns[i])

t_J.save()
t_J.close()


In [84]:
# 给出一组Q的结果
def pre_group_Q(num, ele):
    if ele == 2:
        to_change = np.arange(9.3, max_Q[ele], (max_Q[ele] - 9.3)/num) + (max_Q[ele] - 9.3)/num
    else:
        to_change = np.arange(0, max_Q[ele], max_Q[ele]/num) + max_Q[ele]/num
    res = pd.DataFrame(data=np.zeros((5, num)),columns=to_change)
    for i in range(num):
        data_temp = data_Q_ana_rea.copy()
        data_temp.iloc[:,ele] = to_change[i]
        res.iloc[:,i] = pre_res_Q(data_temp)
    return res

In [85]:
# Q的结果
t_Q = pd.ExcelWriter("问题三铅钡的敏感性分析.xlsx")
for i in range(data_Q_ana_rea.shape[1]):
    temp = pre_group_Q(20, i)
    temp.to_excel(t_Q, data_Q_ana_rea.columns[i])

t_Q.save()
t_Q.close()